# check stft and fft

In [ ]:
import os
import numpy as np

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
import torch
import torch.nn.functional as F

In [ ]:
from scipy.signal import stft
from scipy.fftpack import fft, fftshift

In [ ]:
import h5py
import tqdm
from cplxpaper.musicnet import MusicNetHDF5

In [ ]:
from ipywidgets import widgets

<br>

## Visualizer

In [ ]:
h5_in = h5py.File("./data/musicnet_11khz_train.h5", "r")
dataset = MusicNetHDF5(h5_in, window=4096, stride=512, at=None, resident=True)

In [ ]:
def float_slider(min, max, step, value=None, continuous_update=False):
    layout = widgets.Layout(min_width='500px', display='flex')
    value = min if value is None else min
    return widgets.FloatSlider(min=min, max=max, step=step,value=value,
                               continuous_update=continuous_update, layout=layout)

def int_slider(min, max, step, value=None, continuous_update=True):
    layout = widgets.Layout(min_width='500px', display='flex')
    value = min if value is None else min
    return widgets.IntSlider(min=min, max=max, step=step,value=value,
                             continuous_update=continuous_update, layout=layout)


def update_w_sample(*args):
    beg, end = dataset.limits[w_keys.value]
    w_sample.max = end - beg
    w_sample.value = min(end - beg, max(0, w_sample.value))

w_keys = widgets.Dropdown(options=dir(dataset))
w_sample = int_slider(0, 1, 1)

w_keys.observe(update_w_sample, 'value')
update_w_sample()

In [ ]:
def draw(key, ix):
    fs = 11000
    beg, end = dataset.limits[key]
    data, labels = dataset[ix + beg]
    
    t0 = ix * dataset.stride / fs

    f, t, z = stft(data, fs=fs, nperseg=120, noverlap=60, axis=-1)

    fig, ax = plt.subplots(1, 1, figsize=(12, 3))

    ax.pcolormesh(t0 + t, f, abs(z), vmin=0, cmap=plt.cm.jet)
    ax.set_title(f'STFT Magnitude {key}')
    ax.set_ylabel('Frequency [Hz]')
    ax.set_xlabel('Time [sec]')

    plt.show()

In [ ]:
widgets.interact(draw, key=w_keys, ix=w_sample);

<hr>

In [ ]:
feed = torch.utils.data.DataLoader(dataset, batch_size=512, shuffle=False,
                                   pin_memory=True)

In [ ]:
for bx, by in tqdm.tqdm(feed):
    pass

## Test random access to HDF5 file

Not applicable if `resident=True`.

Compare random access to objects and labels (NLCS)

In [ ]:
cache = dataset
objects, labels, window = cache.objects, cache.labels, cache.window

Prepare indices and pointers

In [ ]:
lengths = np.r_[list(map(len, objects))]
indptr = np.r_[0, lengths.cumsum()]

indices = np.random.randint(indptr[-1], size=1000)

lookup = indptr.searchsorted(indices, side="right") - 1
assert np.all(0 <= lookup) and np.all(lookup < len(objects))

ix_access = np.minimum(indices - indptr[lookup], lengths[lookup] - window)

### Data access

Measure copy and indexing

In [ ]:
out = np.empty(window)
zeros = np.zeros(window)

In [ ]:
%%timeit -n 100
for key, ix in zip(lookup, ix_access):
    obj = objects[key]
    out[:] = zeros[:]

In [ ]:
%%timeit -n 10
for key, ix in zip(lookup, ix_access):
    obj = objects[key]
    out[:] = obj[ix:ix+window]

### Label access

In [ ]:
out = np.empty(84)

In [ ]:
%%timeit -n 1000
for key, ix in zip(lookup, ix_access):
    lab = labels[key]
    zeros = np.zeros(84)
    pass
    out[:] = zeros[:]

In [ ]:
%%timeit -n 100
for key, ix in zip(lookup, ix_access):
    lab = labels[key]
    zeros = np.zeros(84)
    for a, b, i in lab.find_overlap(ix, ix+1):
        zeros[i] = 1
    out[:] = zeros[:]

In [ ]:
%%timeit -n 100
for key, ix in zip(lookup, ix_access):
    lab = labels[key]
    ind = [i for _, _, i in lab.find_overlap(ix, ix+1)]
    out[:] = np.bincount(ind, minlength=84) > 0

<hr>

In [ ]:
dir(cache)

In [ ]:
for key in dir(cache):
    result = cache[slice(*cache.limits[key])]
    assert np.allclose(cache[key], result)

In [ ]:
%%timeit
chunk = cache['id_1759']

In [ ]:
%%timeit
result = cache[slice(*cache.limits['id_1759'])]

<br>